# What is lazy evaluation?

call when you need :)

In [1]:
import functools

In [3]:
class lazy_property:
    def __init__(self, function):
        self.function = function
        functools.update_wrapper(self, function)
        
    def __get__(self, obj, type_):
        if obj is None:
            return self
        val = self.function(obj)
        obj.__dict__[self.function.__name__] = val
        return val

In [4]:
#TODO: work on it
Source = "https://github.com/faif/python-patterns/blob/master/patterns/creational/lazy_evaluation.py"